In [15]:
from my_utils import tile_processing as tp

from stardist.matching import matching_dataset as scorer
import os
from tqdm import tqdm
import pandas as pd
from scipy.io import loadmat

In [16]:
model_name = 'HN_CPM17'
base_folder = r"\\10.99.68.53\Digital pathology image lib\_Image libraries for training\2023-05-09 Published HE Nuclei Datasets"
path_ints = os.path.join(base_folder, "Interpolations")
path_segs = os.path.join(base_folder, "Segmentations " + model_name)
interpolations = os.listdir(path_ints)

In [17]:
taus = [0.5]
columns = ['Model', 'Interpolation', 'Dataset', 'Tile', 'Tau', 'TP', 'FP', 'FN', 'Precision', 'Recall', 'F1-Score', 'Panoptic Quality']
df_scores = pd.DataFrame(columns=columns)

In [18]:
def write_row(column_names, vals, df):
    row = pd.DataFrame(pd.Series(index=column_names)).T
    for c, col_name in enumerate(column_names):
        row[col_name] = vals[c]
    return pd.concat([df, row], ignore_index=True, axis=0)

In [19]:
for interpolation in tqdm(interpolations, desc='Interpolation'):
# for interpolation in [interpolations[0]]:
    path_datasets = os.path.join(path_ints, interpolation)
    path_datasets_HN = os.path.join(path_segs, interpolation)
    for dataset in os.listdir(path_datasets):
    # for dataset in [os.listdir(path_datasets)[0]]:
        path_gts = os.path.join(os.path.join(path_datasets, dataset), r'masks')
        path_preds = os.path.join(os.path.join(path_datasets_HN, dataset), r'images\mat')
        # pred = loadmat('your_file.mat')['inst_map']
        tile_set = tp.TileSetReader(path_gts, '.tif').tile_set
        for i, tilename in enumerate(tile_set[0]):
            gt = tile_set[1][i]
            pred = loadmat(os.path.join(path_preds, tilename + '.mat'))['inst_map']
            gt, pred = tp.remove_margin_objects(gt, 5), tp.remove_margin_objects(pred, 5)
            for tau in taus:
                scores = scorer([gt], [pred], thresh=tau, show_progress=False)._asdict()
                values = [model_name, interpolation, dataset, tilename, tau,
                          scores['tp'], scores['fp'], scores['fn'],
                          scores['precision'], scores['recall'], scores['f1'], scores['panoptic_quality']]
                df_scores = write_row(columns, values, df_scores)

Interpolation: 100%|██████████| 26/26 [06:07<00:00, 14.13s/it]


In [20]:
df_scores

,Model,Interpolation,Dataset,Tile,Tau,TP,FP,FN,Precision,Recall,F1-Score,Panoptic Quality
0,HN_CPM17,L - 20x - 0_,CoNSeP,test_1,0.5,88,58,607,0.602740,0.126619,0.209275,0.145972
1,HN_CPM17,L - 20x - 0_,CoNSeP,test_10,0.5,79,74,561,0.516340,0.123438,0.199243,0.139893
2,HN_CPM17,L - 20x - 0_,CoNSeP,test_11,0.5,79,120,787,0.396985,0.091224,0.148357,0.099463
3,HN_CPM17,L - 20x - 0_,CoNSeP,test_12,0.5,31,45,289,0.407895,0.096875,0.156566,0.104487
4,HN_CPM17,L - 20x - 0_,CoNSeP,test_13,0.5,119,184,1041,0.392739,0.102586,0.162679,0.111641
...,...,...,...,...,...,...,...,...,...,...,...,...
2621,HN_CPM17,R - 40x - 3_,JHU,Skin_train_1,0.5,173,72,33,0.706122,0.839806,0.767184,0.541617
2622,HN_CPM17,R - 40x - 3_,JHU,Skin_train_2,0.5,39,105,11,0.270833,0.780000,0.402062,0.280387
2623,HN_CPM17,R - 40x - 3_,JHU,Skin_train_3,0.5,384,175,284,0.686941,0.574850,0.625917,0.445239
2624,HN_CPM17,R - 40x - 3_,JHU,Skin_train_4,0.5,220,51,112,0.811808,0.662651,0.729685,0.569663


In [21]:
out_folder = r"\\10.99.68.53\Digital pathology image lib\_Image libraries for training\2023-05-09 Published HE Nuclei Datasets\Segmentation Scores on Interpolations"
df_scores.to_csv(os.path.join(out_folder, f'{model_name}.csv'), index=False)